In [1]:
import numpy as np
import pandas as pd
import time

# Hw01
## 作業目標：
1. 熟悉 Numpy, Pandas 的操作，並比較兩者的處理速度
## 作業重點：
1. 使用 Numpy, Pandas 分別計算 “homework.csv” 中每個 feature 的平均值、中位數、最大值、最小值，並比較兩者運算時間
2. 利用以下方式篩選離群值(option)：
* feature 的平均值組成一個高維向量 Vec
* 計算每個 sample 與 Vec 的距離
* 計算距離的平均值、標準差，將距離平均值超過三個標準差的點當作離群值刪除
* __請將刪除離群值後的 dataframe 取名為 df_after_delete__

In [2]:
# 讀取 homework.csv
""" Your Code Here: 讀取 homework.csv"""
df = pd.read_csv("homework.csv")
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79
0,-0.037418,-0.423133,-1.066878,0.463263,-0.130916,-1.153970,-0.598449,-0.327957,-0.007907,-0.464428,...,0.222763,-1.229966,-0.667184,-0.102731,-1.310584,-0.702787,2.059981,0.608418,-0.020808,1.783352
1,0.818413,-1.120974,-0.581785,-1.280338,-0.713844,1.698914,-0.832480,-0.313192,-1.036266,-1.098621,...,-0.445329,-0.693601,0.795167,-1.051140,1.285994,0.653626,1.396237,0.080399,1.377073,-0.794819
2,-2.090596,-0.868769,-1.778862,0.666989,-1.357663,0.433514,-1.617890,-1.220858,-0.566980,-1.237635,...,1.152130,-0.058108,-0.105104,1.080918,0.924454,-0.019370,-0.948623,-0.981408,-0.251559,0.345169
3,-0.289783,1.151368,1.105835,-0.309009,0.186021,0.625453,-1.357677,0.308579,-0.380738,1.335565,...,-0.683605,0.905133,1.328116,0.529399,-0.443334,1.276279,2.503925,0.613427,0.358716,-0.958212
4,0.393337,-1.279473,-0.380730,-3.059202,-1.027667,0.121893,0.088561,-0.444649,0.643246,-0.258456,...,-0.030803,-1.994453,-0.304141,-1.626619,-0.737330,0.276077,-0.052384,0.645169,-0.728073,-0.051508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.169373,-1.659422,1.407234,0.625086,-1.779651,0.684514,-0.564882,-0.159244,0.861659,0.476870,...,-0.226039,1.396990,0.559320,1.195908,0.486600,-1.544905,-1.052251,1.152265,-0.603716,1.214425
99996,0.151488,-0.419368,0.077559,-0.997915,-1.871504,-2.338928,-1.914960,-2.855910,0.738012,-0.174478,...,-1.465184,0.402564,0.220336,-0.065954,1.597364,0.512378,-0.935301,0.152723,-0.134673,-0.591142
99997,1.208199,-1.139107,0.074972,0.371717,0.176787,-0.523450,1.561540,1.371943,-1.379962,-1.025441,...,1.928941,-0.331970,0.573593,0.163113,1.672773,-2.370009,0.746834,1.793470,1.268034,-0.520627
99998,0.038288,0.784779,-0.557020,2.462814,1.129594,-1.292801,-0.650390,0.715064,-1.328429,0.120543,...,-0.723951,0.122555,-1.811966,-0.859171,0.828031,0.699950,-1.044509,-1.729967,1.211119,-0.157613


In [3]:
df.describe()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-0.000982,-0.001772,-0.000067,0.001260,0.004450,0.002382,-0.002804,-0.003068,-0.001736,-0.000383,...,0.001819,0.001489,-0.004019,-0.000977,0.003345,0.003522,-0.002019,-0.003716,0.004458,0.004140
std,0.997387,0.999677,0.999423,1.000450,1.002278,0.998099,1.002785,0.997109,0.999793,1.002520,...,0.993507,0.995408,0.999268,0.999840,1.003151,1.000187,1.001023,0.997058,1.003886,1.002848
min,-4.698802,-4.485786,-4.403587,-4.488550,-4.576555,-4.671108,-4.388425,-4.379263,-4.280279,-4.101829,...,-4.207474,-4.474170,-4.190348,-3.936072,-4.560416,-4.533724,-4.123944,-4.597586,-4.159297,-4.491868
25%,-0.676371,-0.674310,-0.678171,-0.675814,-0.673363,-0.667494,-0.681587,-0.676339,-0.677416,-0.678124,...,-0.666864,-0.671852,-0.677088,-0.670269,-0.673437,-0.669786,-0.678369,-0.675646,-0.672983,-0.673700
50%,-0.002080,-0.001812,-0.005066,0.003780,0.005583,0.000566,-0.004926,-0.000537,-0.007138,0.002882,...,0.002379,0.001827,-0.003030,0.000725,0.002361,0.001356,-0.003176,-0.007174,0.005047,0.004556
75%,0.673900,0.669970,0.675509,0.672265,0.683034,0.680560,0.672093,0.671895,0.667823,0.673553,...,0.672958,0.673446,0.669429,0.674588,0.683595,0.674703,0.672470,0.668894,0.682869,0.680835
max,4.292971,4.340301,4.915044,4.077389,4.056038,4.390056,4.060047,4.351157,4.647597,4.436166,...,4.270831,4.266420,4.059832,4.556041,4.221023,3.979760,4.348522,4.159195,4.382344,4.869825


## 1. 使用 Numpy, Pandas 分別計算 “homework.csv” 中每個 feature 的平均值、中位數、最大值、最小值，並比較兩者運算時間

In [4]:
# -------------- Numpy --------------
arr = np.array(df, dtype = 'float')

start = time.time()

""" Your Code Here: 用 Numpy 計算 “homework.csv” 中每個 feature 的平均值、中位數、最大值、最小值，並比較兩者運算時間"""
mean_1 = np.mean(df,axis=0)
median_1 = np.median(df,axis=0)
max_1 = np.max(df,axis=0)
min_1 = np.min(df,axis=0)


end = time.time()
print(f'Pandas has done in {(end - start):.4f} sec.')

# -------------- Pandas --------------
start = time.time()

""" Your Code Here: 用 Pandas 計算 “homework.csv” 中每個 feature 的平均值、中位數、最大值、最小值，並比較兩者運算時間"""
mean_2 = df.mean()
median_2 = df.median()
max_2 = df.max()
min_2 = df.min()

end = time.time()
print(f'Pandas has done in {(end - start):.4f} sec.')

Pandas has done in 0.1227 sec.
Pandas has done in 0.1217 sec.


## 2. 利用以下方式篩選離群值(option)：
* feature 的平均值組成一個高維向量 Vec
* 計算每個 sample 與 Vec 的距離
* 計算距離的平均值、標準差，將距離平均值超過三個標準差的點當作離群值刪除
* __請將刪除離群值後的 dataframe 取名為 df_after_delete__

In [42]:
# 計算各個 sample 與 Vec 之距離
""" Your Code Here: 計算各個 sample 與 Vec 之距離"""
featureMean = df.mean()
error = df - featureMean

# 計算 distance 的 mean, std, 並篩選離群值
""" Your Code Here: 計算 distance 的 mean, std, 並篩選離群值"""
distance1 = error.mean()+3*error.std()
distance2 = error.mean()-3*error.std()

# 刪除離群值
""" Your Code Here: 刪除離群值"""
a = True in (error > distance1 ) or (error < distance2)
indexs = np.where(a.values == True)[0]
df_after_delete = df.drop(index=indexs)
df_after_delete

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79
0,-0.037418,-0.423133,-1.066878,0.463263,-0.130916,-1.153970,-0.598449,-0.327957,-0.007907,-0.464428,...,0.222763,-1.229966,-0.667184,-0.102731,-1.310584,-0.702787,2.059981,0.608418,-0.020808,1.783352
1,0.818413,-1.120974,-0.581785,-1.280338,-0.713844,1.698914,-0.832480,-0.313192,-1.036266,-1.098621,...,-0.445329,-0.693601,0.795167,-1.051140,1.285994,0.653626,1.396237,0.080399,1.377073,-0.794819
2,-2.090596,-0.868769,-1.778862,0.666989,-1.357663,0.433514,-1.617890,-1.220858,-0.566980,-1.237635,...,1.152130,-0.058108,-0.105104,1.080918,0.924454,-0.019370,-0.948623,-0.981408,-0.251559,0.345169
3,-0.289783,1.151368,1.105835,-0.309009,0.186021,0.625453,-1.357677,0.308579,-0.380738,1.335565,...,-0.683605,0.905133,1.328116,0.529399,-0.443334,1.276279,2.503925,0.613427,0.358716,-0.958212
5,0.141568,-0.142398,-0.223417,-0.135351,-0.123956,-0.580145,1.710353,-1.722192,1.846158,0.457103,...,-1.517435,-0.804391,-0.470260,0.308841,-0.047055,-1.034696,-0.112808,-0.402497,0.923015,1.830315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.169373,-1.659422,1.407234,0.625086,-1.779651,0.684514,-0.564882,-0.159244,0.861659,0.476870,...,-0.226039,1.396990,0.559320,1.195908,0.486600,-1.544905,-1.052251,1.152265,-0.603716,1.214425
99996,0.151488,-0.419368,0.077559,-0.997915,-1.871504,-2.338928,-1.914960,-2.855910,0.738012,-0.174478,...,-1.465184,0.402564,0.220336,-0.065954,1.597364,0.512378,-0.935301,0.152723,-0.134673,-0.591142
99997,1.208199,-1.139107,0.074972,0.371717,0.176787,-0.523450,1.561540,1.371943,-1.379962,-1.025441,...,1.928941,-0.331970,0.573593,0.163113,1.672773,-2.370009,0.746834,1.793470,1.268034,-0.520627
99998,0.038288,0.784779,-0.557020,2.462814,1.129594,-1.292801,-0.650390,0.715064,-1.328429,0.120543,...,-0.723951,0.122555,-1.811966,-0.859171,0.828031,0.699950,-1.044509,-1.729967,1.211119,-0.157613


## --驗證答案--

In [ ]:
ans = pd.read_csv('./data/ans.csv')
judge = not (False in (ans == df_after_delete))
print(f'Your answer is {judge}.')